In [14]:
import SimpleITK as sitk
import pydicom
import numpy as np
import glob, os
from skimage.filters import gaussian
from tqdm import tqdm
import SimpleITK as sitk
from einops.einops import rearrange

In [19]:
sd = 0
files = glob.glob(rf'data/DigitalPhantomCT_SD_{sd}_not_normalized/*')
vof_location = (410, 247, 16)  # start, start, size
aif_location = (123, 251, 8)  # start, start, size
aif_values_rescaled_nosmoothing = [40.88909, 39.572624, 39.66895, 40.150585, 39.861603, 41.17807, 41.17807, 259.03726, 494.58847, 447.00302, 291.8205, 170.57712, 100.96492, 64.71393, 49.847485, 44.93482, 42.494537, 40.150585, 39.508408, 38.320374, 40.95331, 39.572624, 40.150585, 41.24229, 41.402832, 39.829494, 39.219425, 39.37997, 41.659706, 38.994663]
for ix, file in tqdm(enumerate(files[::16])):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    new_aif_value = aif_values_rescaled_nosmoothing[ix]
    new_aif_array = np.tile(np.array(new_aif_value), (8, 8))
    # data[aif_location[0]:aif_location[0] + aif_location[2],
    #            aif_location[1]:aif_location[1] + aif_location[2]] = new_aif_array
    # copy the data back to the original data set
    ds.PixelData = data.tobytes()
    # print('The information of the data set after downsampling: \n')
    # print(ds.pixel_array[aif_location[0]:aif_location[0] + aif_location[2],
    #            aif_location[1]:aif_location[1] + aif_location[2]])
    ds.save_as(file)

30it [00:00, 211.27it/s]


In [20]:

simulated_data_size = 32 * 7
scan_center = 512 // 2
simulated_data_start = scan_center - simulated_data_size // 2
simulated_data_end = scan_center + simulated_data_size // 2
other_files = sorted([file for file in files if file not in files[::16]])

for ix, file in tqdm(enumerate(other_files)):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    phantomcurves = data[simulated_data_start:simulated_data_end,
                         simulated_data_start:simulated_data_end]
    truncate = np.ceil(2 * sd) / sd if sd != 0 else 0
    phantomcurves = gaussian(phantomcurves, sigma=(sd, sd), mode='nearest', truncate=truncate, preserve_range=True).astype(np.int16)
    data[simulated_data_start:simulated_data_end, simulated_data_start:simulated_data_end] = phantomcurves
    ds.PixelData = data.tobytes()
    ds.save_as(file)


450it [00:02, 194.38it/s]


In [12]:
# directory = r"C:\Users\lucasdevries\surfdrive\Projects\ppinn\data\PPINN0_KUDO_SD_0"
# reader = sitk.ImageSeriesReader()
# dicom_names = reader.GetGDCMSeriesFileNames(directory)
# reader.SetFileNames(dicom_names)
# image = reader.Execute()
# image_data = sitk.GetArrayFromImage(image)
# image_data = rearrange(image_data, '(t values) h w -> values t h w', t=30)

In [13]:
# image_data.shape

(16, 30, 512, 512)

In [8]:
# path = r'C:\Users\lucasdevries\surfdrive\Projects\ppinn\data\image_data_sd_2.nii'
# image_data_nii = sitk.ReadImage(path)

In [10]:
# sitk.GetArrayFromImage(image_data_nii).shape

(224, 224, 15, 30)